
# **Multi Agent Deep Q-Learning**

## **Problem Description**

This task involves 4 agents of 2 types navigating a 5x5 grid world to reach a goal position B. An agent must meet an agent of opposite type to exchange secret before they can end the episode by reaching B. Agents cannot exchange secret at goal position B and only one agent is needed to reach B to terminate the episode.

All positions are randomised and can overlap.

## **Chosen contract options**
The following options are chosen
- Nearest opposite type agent (location)
- Off the job training (not used)
- Clock

## **Methodology**

The agent uses Deep Q-learning to learn an optimal policy. The parameters for the agent are as follow

|    **Parameter**    | **Value** |
|:---------------|-----------:|
| gamma (reward discount) | 0.9 |
| optimizer.alpha | 0.001 |
| dqn.hidden_size | [250, 250] |
| epsilon_max | 1.0 |
| epsilon_min | 0.005 |
| eps_decay_final_step | 19900 |
| replay buffer size | 100 |
| batch size | 32 |
| network copy frequency | 80 |

The agent chooses an action with `ε`-greedy policy then stores the experience in replay memory D. When there is enough experience in D, randomly extract a minibatch of experiences and calculate the target value by
$$
\begin{cases}
    y_i = r_i & \text{if done} \\
    y_i = r_i + \gamma \max_{a'} \^{Q}(s'_i, a'_i) & \text{otherwise.}
\end{cases}
$$

Then do one step of learning.

To facilitate convergence, three changes were made besides using the given API and implementing the learning algorithm.

1. **torch.tensors** are used throughout and GPU/MPS acceleration used whenever available
2. maximum of 50 steps can be spent in one episode before resetting 

### **State Space**

The state space is defined by the agent’s position on the grid, the closest location of agent of opposite type (A), and the location of goal (B). Also 1 extra bit is used for whether or not the agent has received the secret. Since our grid is 5 by 5, and we need to use one-hot encoding for machine learning, this translates into 76 bits, 25 for each position / location.

### **Action Space**

The agent can perform one of four actions at any given time:

- **Move North**
- **Move South**
- **Move West**
- **Move East**
- **Stay Still**

These actions move the agent one step in the corresponding direction unless the movement would result in the agent hitting a wall, in which case the agent remains in the same position.

### **Reward Structure**

The reward structure is designed to guide the agent toward efficiently solving the task:

|    **Event**    | **Reward** |
|:---------------|-----------:|
| Receiving Secret (first time) | +20 |
| Reaching Goal with Secret | +50 |
| Every Round | -1 |
| Hitting a wall | -10 |

Multiple rewards may accumulate at each round. For example, if a type 1 agent, not having the secret, moves into a cell occupied by a type 2 agent in a non-goal cell results in a reward of 19. This reward system incentivizes the agent to exchange secret and reach the goal while penalizing unnecessary movements and collisions with walls.

If the agent reaches the goal position without the secret, the reward would be -1 and the game would not terminate.

## **Performance Metrics**

### **ML Loss**
To show the progress of learning wtih Deep Q-Learning, there is a graph showing average MSE loss in each episode. This is the returned value from given skeleton divided by batch_size, then averaged over all learnings, to show the loss per episode.

### **Excess Step**

To visualize the agents' performance, the excess step is calculated by the number of steps taken minus the minimum steps possible in a scenario. In both calculations, we calculate with **Clock** enabled.

During training and testing, we reorder the agents' move order to make sure the agents move according to descending order to the goal position to make sure the minimum steps possible is actually achieveable.


#### **Minimum Step Calculation**

To calculate the minimum number of steps needed, we loop between all possible pairs of agents and solve the subproblem when there were only 2 agents on the grid. We find the minimum steps for each scenario with 2 agents, and the minimum of all scenarios is the number we wanted.

To solve the sub problem, we first observe that the answer must be closely correlated to the max(distance1, distance_2), where distance_1 and distance_2 are the corresponding manhattan distances of the agents and the goal position. This is because by meeting up and going to the goal position, the total distance travelled is simply the larger agent-goal distance. It does not matter whether the other agent decides to wait near goal position or move towards the further agent.

Hence we first calculate the manhattan distances.

If they can meet up without going through the goal position, no extra step is added, otherwise 2 is added for the closer agent to go through the goal and meet at wherever convenient.

Then the larger distance can be reduced by 1 if clock is enabled. And the result is the max of both modified distances.
```
def calculate_min_step_for_two(self, one_pos, two_pos, goal_pos, clock=False):
    one_dist = self.calc_mht_dist(one_pos, goal_pos)
    two_dist = self.calc_mht_dist(two_pos, goal_pos)

    smaller, larger = min(one_dist, two_dist), max(one_dist, two_dist)
    smaller = smaller + (
        0
        if self.line_passing_through_goal_can_cut(one_pos, two_pos, goal_pos)
        else 2
    )
    smaller, larger = min(smaller, larger), max(smaller, larger)

    return max(smaller, larger - (1 if clock else 0))
```

In [1]:
%matplotlib widget

In [2]:
# constants.py

import torch

side = 5
action_size = 5
action_space = [(0, -1), (0, 1), (-1, 0), (1, 0), (0, 0)]

state_size = 5 * 5 * 3 + 1
device = torch.device(
    "cuda:0"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
dtype = torch.float32
# if device.type == "mps" else torch.float64
debug = False

In [3]:
# agent.py

from typing import TYPE_CHECKING, Tuple, List
from abc import abstractmethod, ABC

import numpy as np
import random
import torch

if TYPE_CHECKING:
    pass


class ExpBuffer:
    def __init__(self, max):
        self.max = max
        self.itr = 0
        self.has_reached = False

        self.states = torch.empty((self.max, state_size), dtype=dtype)
        self.actions = torch.empty((self.max,), dtype=dtype)
        self.rewards = torch.empty((self.max,), dtype=dtype)
        self.next_states = torch.empty((self.max, state_size), dtype=dtype)
        self.is_terminals = torch.empty((self.max,), dtype=torch.bool)
        pass

    def insert(self, state, action, reward, next_state, is_terminal):
        self.itr %= self.max
        self.states[self.itr] = state
        self.actions[self.itr] = action
        self.rewards[self.itr] = reward
        self.next_states[self.itr] = next_state
        self.is_terminals[self.itr] = is_terminal
        self.itr += 1

        if self.itr >= self.max:
            self.has_reached = True

    def extract(self, batch_size) -> Tuple[List[List[int]], List[int], List[float]]:
        indices = np.random.randint(
            0, self.max if self.has_reached else self.itr, batch_size
        )
        return (
            self.states[indices],
            self.actions[indices],
            self.rewards[indices],
            self.next_states[indices],
            self.is_terminals[indices],
        )

    def clear(self):
        self.__init__(self.max)

    def __len__(self):
        return len(self.states)


class Agent(ABC):
    def __init__(
        self,
        dqn,
        buffer,
        gamma=0.997,
        batch_size=200,
    ):
        self.dqn = dqn
        self.buffer = buffer
        self.actions = action_space

        # Agent Properties
        self.total_reward = 0
        self.have_secret = False

        # Initialize Q Table for all state-action to be 0
        self.batch_size = batch_size

        # Initialize Learning param
        self.epsilon = 1  # Epsilon is updated at Grid level
        self.gamma = gamma

        self.learning = True

    # ----- Core Functions ----- #
    def choose_action(self, state: torch.tensor, choose_best: bool) -> Tuple[int, int]:
        if (
            not choose_best
            and np.random.rand() < self.epsilon  # Epsilon is updated at Grid level
        ):
            return random.choice(self.actions)
        else:
            # Extract immutable state information
            idx = torch.argmax(self.dqn.get_qvals(state))
            return self.actions[idx]

    def update(
        self,
        state: torch.tensor,
        action: Tuple[int, int],
        reward: int,
        next_state: torch.tensor,
        is_terminal: bool,
    ):
        self.total_reward += reward
        if not self.learning:
            return None, None

        self.buffer.insert(
            state, self.actions.index(action), reward, next_state, is_terminal
        )
        loss = None
        if len(self.buffer) >= self.batch_size:
            states, actions, rewards, next_states, is_terminals = self.buffer.extract(
                self.batch_size
            )
            rewards = rewards.to(device)
            targets = self.gamma * self.dqn.get_maxQ(next_states) + rewards

            # For terminal states, target_val is reward
            indices = is_terminals.nonzero().to(device)
            targets[indices] = rewards[indices]

            loss = self.dqn.train_one_step(states, actions, targets)

        return loss, self.epsilon

    # ----- Public Functions ----- #
    def have_secret_(self, new_value: bool):
        self.have_secret = new_value

    def reset(self):
        self.have_secret = False
        self.total_reward = 0

    def get_total_reward(self):
        return self.total_reward

    def enable_learning(self):
        if not self.learning:
            self.buffer.clear()
        self.learning = True

    def disable_learning(self):
        self.learning = False

    # ----- Private Functions ----- #
    @abstractmethod
    def get_type(self):
        pass

    def is_different_type(self, other: "Agent"):
        return other.get_type() != self.get_type()


class Agent1(Agent):
    def get_type(self):
        return 1


class Agent2(Agent):
    def get_type(self):
        return 2


In [4]:
# given.py

import torch
import copy
import numpy as np

class DQN:
    def __init__(self, state_size, action_size=4):
        l1 = state_size
        l2 = 250
        l3 = 250
        l5 = action_size
        self.model = torch.nn.Sequential(
            torch.nn.Linear(l1, l2),
            torch.nn.ReLU(),
            torch.nn.Linear(l2, l3),
            torch.nn.ReLU(),
            torch.nn.Linear(l3, l5),
        ).to(device)

        self.model2 = copy.deepcopy(self.model).to(device)
        self.model2.load_state_dict(self.model.state_dict())
        self.loss_fn = torch.nn.MSELoss()
        self.learning_rate = 1e-3
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.learning_rate
        )

    # The function "update_target" copies the state of the prediction network to the target network. You need to use this in regular intervals.
    def update_target(self):
        self.model2.load_state_dict(self.model.state_dict())

    # The function "get_qvals" returns a numpy list of qvals for the state given by the argument based on the prediction network.
    def get_qvals(self, state):
        q_values = self.model(state.to(device)).to(device)
        return q_values

    # The function "get_maxQ" returns the maximum q-value for the state given by the argument based on the target network.
    def get_maxQ(self, state):
        return torch.max(self.model2(state.to(device)), dim=1).values.float()

    # The function "train_one_step_new" performs a single training step.
    # It returns the current loss (only needed for debugging purposes).
    # Its parameters are three parallel lists: a minibatch of states, a minibatch of actions,
    # a minibatch of the corresponding TD targets and the discount factor.
    def train_one_step(self, states, actions, targets):
        # state1_batch = torch.cat([torch.from_numpy(s).float() for s in states])
        state1_batch = states.to(device)
        action_batch = actions.to(device)
        targets = targets.to(device)
        # print(action_batch.shape)
        # print(state1_batch.shape)
        Q1 = self.model(state1_batch)
        X = Q1.gather(dim=1, index=action_batch.long().unsqueeze(dim=1)).squeeze()
        Y = targets.clone().detach().to(device).float()
        loss = self.loss_fn(X, Y)
        # print(loss)
        self.optimizer.zero_grad()
        # torch.nn.utils.clip_grad_norm_(self.model.parameters(), 5000)
        loss.backward()

        # total_norm = 0
        # for p in self.model.parameters():
        #     param_norm = p.grad.data.norm(2)
        #     total_norm += param_norm.item() ** 2
        # total_norm = total_norm ** (1.0 / 2)
        # print(total_norm)

        self.optimizer.step()
        return loss.item() / len(X)

    def save(self, prefix):
        torch.save(self.model.state_dict(), f"{prefix}_1.pth")
        torch.save(self.model2.state_dict(), f"{prefix}_2.pth")

    def load(self, prefix):
        try:
            self.model.load_state_dict(
                torch.load(f"{prefix}_1.pth", weights_only=False, map_location=device)
            )
            self.model2.load_state_dict(
                torch.load(f"{prefix}_2.pth", weights_only=False, map_location=device)
            )
        except:
            print("load state_dict failed")


In [5]:
# storage.py

from multiprocessing import Array
from typing import List

class Storage:
    def __init__(self, max_itr: int):
        self.itr = 0
        self.max_itr = max_itr

        # TODO: max_itr could be dynamic
        self.iterations = Array("i", range(max_itr))
        self.excess_step = Array("i", max_itr)
        self.epsilon = Array("f", max_itr)
        self.test_loss = []
        self.ml_losses = []

        self.step_count_hist = Array("i", 51)
        self.excess_step_hist = Array("i", 51)

    def reset_counter(self):
        self.itr = 0

    def append_excess_epsilon(self, excess_step: int, epsilon: float):
        if self.itr >= self.max_itr:
            self.itr = 0
        self.excess_step[self.itr] = excess_step
        self.epsilon[self.itr] = epsilon
        self.itr += 1

    def append_step_count_hist(self, step_count: int):
        self.step_count_hist[step_count] += 1

    def append_excess_step_hist(self, excess_step: int):
        self.excess_step_hist[excess_step] += 1

    def append_test_loss(self, test_loss: int):
        self.test_loss.append(test_loss)

    def reset_test_loss(self):
        self.test_loss = []

    def append_ml_losses(self, ml_losses: float):
        self.ml_losses.append(ml_losses)

    def get_all(self):
        return self.iterations, self.losses, self.epsilon, self.test_loss


In [6]:
# view.py

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation

from matplotlib.widgets import Button
from typing import Tuple, TypeAlias, TYPE_CHECKING, List
from abc import ABC, abstractmethod

Coordinates: TypeAlias = Tuple[float, float, float, float]


class IVisual(ABC):

    # Getting Info
    @abstractmethod
    def get_agent_info(self) -> List[Tuple[Tuple[int, int], bool]]:
        pass

    @abstractmethod
    def get_total_reward(self) -> int:
        pass

    @abstractmethod
    def get_min_step(self) -> int:
        pass

    @abstractmethod
    def get_size(self) -> Tuple[int, int]:
        pass

    @abstractmethod
    def get_goal_positions(self) -> List[Tuple[int, int]]:
        pass

    @abstractmethod
    def get_agent_positions(self) -> List[Tuple[int, int]]:
        pass

    @abstractmethod
    def has_ended(self) -> bool:
        pass

    # Functions
    @abstractmethod
    def toggle_auto_reset(self):
        pass

    @abstractmethod
    def next(self):
        pass

    @abstractmethod
    def reset(self):
        pass

    # Trainings/Testing
    @abstractmethod
    def train(self, itr=1):
        pass

    @abstractmethod
    def test(self, itr=1):
        pass


class Visualization:
    def __init__(self, storage: "Storage", grid: "Grid"):
        self.storage = storage
        self.grid = grid
        self.fig, self.ax = plt.subplots()
        self.animating = False

    def show(self):
        assert self.grid is not None

        self.add_ui_elements()
        self.fig.canvas.mpl_connect("close_event", self.on_close)
        self.ani = animation.FuncAnimation(
            self.fig, self.draw, frames=self.frames, interval=200, save_count=100
        )
        self.animating = True
        plt.show()

    def get_info(self):
        agent_info = self.grid.get_agent_info()
        step_count = self.grid.get_step_count()
        min_step = self.grid.get_min_step()
        return agent_info, step_count, min_step

    def frames(self):
        while True:
            if self.animating:
                self.grid.next()
                yield self.get_info()
            else:
                yield self.get_info()

    # ----- ----- ----- ----- Drawing Functions  ----- ----- ----- ----- #

    def draw(self, args):
        agent_info, step_count, min_step = args

        self.ax.clear()
        self.draw_grid()
        self.draw_agent(agent_info)

        self.step_count.set_text(f"Step: {step_count}")
        self.min_step.set_text(f"Min Step: {min_step}")

        # Check if the environment is terminal
        if self.grid.has_ended():
            self.draw_complete()

        # Early return if animating, since animation automatically refreshes canvas
        if self.animating:
            return

        self.fig.canvas.draw()

    def draw_grid(self):
        width, height = self.grid.get_size()
        for x in range(width):
            for y in range(height):
                rect = patches.Rectangle(
                    (x, y), 1, 1, linewidth=1, edgecolor="black", facecolor="white"
                )
                self.ax.add_patch(rect)
        self.ax.set_xlim(0, width)
        self.ax.set_ylim(height, 0)
        self.ax.set_aspect("equal")

        # Move x-axis labels to the top
        self.ax.xaxis.set_label_position("top")
        self.ax.xaxis.tick_top()

        # Draw target
        tx, ty = self.grid.get_goal_positions()
        target_patch = patches.Rectangle(
            (tx, ty), 1, 1, linewidth=1, edgecolor="black", facecolor="green"
        )
        self.ax.add_patch(target_patch)

    def draw_agent(self, info):
        # Draw agent
        for idx, (pos, type, has_item, step_count) in enumerate(info):
            dx = [0, 0.5, 0, 0.5][idx]
            dy = [0, 0, 0.5, 0.5][idx]
            ax, ay = pos

            if type == 1:
                agent_color = "red" if has_item else "pink"
            if type == 2:
                agent_color = "blue" if has_item else "cyan"

            agent_patch = patches.Rectangle(
                (ax + dx, ay + dy),
                0.5,
                0.5,
                linewidth=1,
                edgecolor="black",
                facecolor=agent_color,
            )
            self.ax.add_patch(agent_patch)
            self.ax.text(
                ax + dx + 0.17,
                ay + dy + 0.33,
                step_count,
                c="yellow",
                ma="center",
                size="large",
                weight="bold",
                # backgroundcolor="white",
            )

    def draw_complete(self):
        self.ax.text(
            0.5,
            0.5,
            "Complete",
            horizontalalignment="center",
            verticalalignment="center",
            transform=self.ax.transAxes,
            fontsize=20,
            color="red",
        )

    # ----- ----- ----- ----- Render UI Element  ----- ----- ----- ----- #

    def add_ui_elements(self):
        self.init_buttons()
        self.init_text()

    def init_buttons(self):
        self.toggle_anim_btn = None
        self.toggle_auto_reset_btn = None

        btn_template = [
            ("Next Step", self.on_next),
            ("Reset", self.on_reset),
            ("Anim\nOn", self.on_toggle_anim, "toggle_anim_btn"),
            ("Auto Reset\nOn", self.on_auto_reset, "toggle_auto_reset_btn"),
        ]
        self.buttons = []
        x, y, w, h = 0.85, 0.01, 0.12, 0.075
        for template in btn_template:
            ref = None
            try:
                label, cb, ref = template
            except:
                label, cb = template
            self.buttons.append(self.add_button([x, y, w, h], label, cb))
            y += 0.1
            if ref:
                setattr(self, ref, self.buttons[-1])

    def init_text(self):
        # Add text box for cumulative reward
        self.step_count = self.add_text([0.01, 0.01, 0.2, 0.075], f"Step: 0")

        # Add text box for max reward
        self.min_step = self.add_text(
            [0.25, 0.01, 0.2, 0.075],
            f"Min Step: {self.grid.get_min_step()}",
        )

    def add_button(self, coordinates: Coordinates, text, on_click):
        axis = plt.axes(coordinates)
        button = Button(axis, text)
        button.on_clicked(on_click)

        return button

    def add_text(self, coordinates: Coordinates, text):
        axis = plt.axes(coordinates)
        textbox = axis.text(
            0.5,
            0.5,
            text,
            horizontalalignment="center",
            verticalalignment="center",
            transform=axis.transAxes,
            fontsize=12,
        )
        axis.axis("off")
        return textbox

    # ----- ----- ----- ----- Event Handlers  ----- ----- ----- ----- #

    def on_close(self, e):
        self.ani.event_source.stop()

    def on_auto_reset(self, event):
        is_on = self.grid.toggle_auto_reset()
        if is_on:
            self.toggle_auto_reset_btn.label.set_text("Auto Reset\nOn")
        else:
            self.toggle_auto_reset_btn.label.set_text("Auto Reset\nOff")
        plt.show()

    def on_toggle_anim(self, event):
        if self.animating:
            self.toggle_anim_btn.label.set_text("Anim\nOff")
        else:
            self.toggle_anim_btn.label.set_text("Anim\nOn")

        self.animating = not self.animating
        plt.show()

    def on_reset(self, event):
        self.grid.reset()
        self.draw(self.get_info())

    def on_next(self, e):
        self.grid.next()
        self.draw(self.get_info())


class Graph:
    def __init__(self, storage: "Storage", keys=[]):
        self.storage = storage
        self.keys = keys

        self.fig, self.axs = plt.subplots(1, len(self.keys))
        if len(self.keys) == 1:
            self.axs = [self.axs]
        self.ani = animation.FuncAnimation(
            self.fig, self.draw, frames=self.frames, interval=100, save_count=100
        )

    def frames(self):
        while True:
            yield None

    def show(self):
        plt.show()

    # Compulsory unused argument
    def draw(self, args):
        for i, k in enumerate(self.keys):
            self.plot(
                self.axs[i],
                range(len(getattr(self.storage, k))),  # self.storage.iterations
                getattr(self.storage, k),
                k,
            )

    def plot(self, ax, itr, values, key):
        ax.plot(itr, values, color="blue", label="Loss")
        ax.set_title(key.title())
        ax.set_xlabel("Iteration")
        ax.set_ylabel(key.title())


In [7]:
# grid.py

import torch
from abc import ABC, abstractmethod
from typing import TYPE_CHECKING, List, Tuple, Dict, Set

import random

import datetime

class Controller:
    # Iterate by number of games
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.auto_reset = True

    def toggle_auto_reset(self):
        self.auto_reset = not self.auto_reset
        return self.auto_reset

    def next(self):
        if self.has_ended() and self.auto_reset:
            self.reset()
        else:
            self.step(is_testing=True)
        return


class Trainer:
    def __init__(self, storage, **kwargs):
        super().__init__(**kwargs)
        self.storage = storage
        self.learners = [1, 2]

    def train(
        self,
        itr=1,
        upd_freq=100,
        eps_min=5e-2,
        eps_decay_final_step=15e3,
        max_grad_norm=5e3,
        dqn1=None,
        dqn2=None,
    ):
        def calc_eps(start_eps, end_eps, step, final_step):
            return (
                start_eps + (end_eps - start_eps) * min(step, final_step) / final_step
            )

        start = datetime.datetime.now()
        print(f"Start Time: {start}")

        self.enable_learning(agent_type=1)
        self.enable_learning(agent_type=2)

        for i in range(itr):
            # Copy Q to Q_hat
            if i % upd_freq == 0:
                for dqn in [dqn1, dqn2]:
                    dqn.update_target()

            # Decay epsilon linearly, reaching eps_min at eps_decay_final_step
            epsilon = calc_eps(1, eps_min, i, eps_decay_final_step)
            for agent in self.agents:
                agent.epsilon = epsilon

            self.reset()
            (excess_step, reward, epsilon, ml_losses, step_count) = self.play_one_game(
                is_testing=False
            )

            self.storage.append_ml_losses(sum(ml_losses) / len(ml_losses))
            self.storage.append_excess_epsilon(excess_step, epsilon)

            if (i + 1) % 100 == 0:
                print(
                    f"Epoch: {i+1}/{itr} -- Time Elapsed: {datetime.datetime.now() - start}"
                )
        return self.storage.ml_losses

    def small_test(self, itr=1):
        total_loss = 0
        total_step_count = 0
        success_count = 0

        self.disable_learning(agent_type=1)
        self.disable_learning(agent_type=2)

        for i in range(itr):
            self.reset()
            excess_step, reward, eps, ml_losses, step_count = self.play_one_game(
                is_testing=True
            )
            self.storage.append_excess_step_hist(excess_step)
            if step_count < 15:
                success_count += 1
            total_loss += excess_step
            total_step_count += step_count
            if (i + 1) % 100 == 0:
                print(
                    f"Ep {i + 1}: Avg = {total_step_count / (i + 1)}; Excess = {total_loss / (i + 1)}; Success = {success_count/(i + 1)}"
                )
        return total_loss / itr

    def test(self, max_itr=None):
        itr = 0

        self.disable_learning(agent_type=1)
        self.disable_learning(agent_type=2)

        possible_loc = [(x, y) for x in range(side) for y in range(side)]
        total_step_count = 0
        excess_step_count = 0
        success_count = 0
        count = 0

        for goal_pos in possible_loc:
            for p1 in possible_loc:
                for p2 in possible_loc:
                    for p3 in possible_loc:
                        for p4 in possible_loc:
                            if max_itr is not None and itr >= max_itr:
                                return
                            self.agent_positions = [p1, p2, p3, p4]
                            self.goal_pos = goal_pos
                            self.reset(random_pos=False)

                            (excess_step, reward, epsilon, ml_loss, step_count) = (
                                self.play_one_game(is_testing=True)
                            )

                            self.storage.append_step_count_hist(step_count)
                            self.storage.append_excess_step_hist(excess_step)

                            total_step_count += step_count
                            excess_step_count += excess_step
                            if step_count < 15:
                                success_count += 1
                            count += 1

                            if count % 100 == 0:
                                print(
                                    f"Ep {count}: Avg = {total_step_count / count}; Excess = {excess_step_count / count}; Success = {success_count / count}"
                                )
                            itr += 1

    def enable_learning(self, agent_type):
        if agent_type not in self.learners:
            self.learners.append(agent_type)
        for a in [a for a in self.agents if a.get_type() == agent_type]:
            a.enable_learning()

    def disable_learning(self, agent_type):
        if agent_type in self.learners:
            self.learners.remove(agent_type)
        for a in [a for a in self.agents if a.get_type() == agent_type]:
            a.disable_learning()

    def play_one_game(self, is_testing=False, **kwargs):
        max_step_count = 50
        self.step_count = 1
        ml_losses = []
        epsilons = []
        while not self.has_ended() and self.step_count < max_step_count:
            ml_loss, epsilon = self.step(is_testing)

            if ml_loss is not None:
                ml_losses.append(ml_loss)
            if epsilon is not None:
                epsilons.append(epsilon)
        total_reward = sum(map(lambda a: a.get_total_reward(), self.agents))
        loss = self.step_count - self.min_step
        return (
            loss,
            total_reward,
            None if len(epsilons) == 0 else epsilons[-1],
            ml_losses,
            self.step_count,
        )


class Visual:
    def get_agent_info(self) -> List[Tuple[Tuple[int, int], int, bool]]:
        """
        Output: List of
                - Tuple of:
                    - coordinate: (int, int)
                    - type: int (1 or 2)
                    - have_secret: bool
        """
        agent_types = map(lambda agent: agent.get_type(), self.agents)
        have_secrets = map(lambda agent: agent.have_secret, self.agents)
        step_counts = [
            self.step_count if idx < self.idx else self.step_count - 1
            for idx in range(len(self.agents))
        ]
        return list(
            zip(self.get_agent_positions(), agent_types, have_secrets, step_counts)
        )

    def get_total_reward(self):
        return sum(map(lambda a: a.get_total_reward(), self.agents))

    def get_min_step(self):
        return self.min_step

    def get_size(self):
        return self.width, self.height

    def get_goal_positions(self):
        return self.goal_pos

    def get_agent_positions(self):
        return self.agent_positions

    def has_ended(self) -> bool:
        return self.goal_reached

    def get_step_count(self) -> int:
        # Incremented when last agent made a move
        return self.step_count


class GridUtil:
    def calculate_min_step_for_two(self, one_pos, two_pos, goal_pos, clock=False):
        one_dist = self.calc_mht_dist(one_pos, goal_pos)
        two_dist = self.calc_mht_dist(two_pos, goal_pos)

        smaller, larger = min(one_dist, two_dist), max(one_dist, two_dist)
        smaller = smaller + (
            0
            if self.line_passing_through_goal_can_cut(one_pos, two_pos, goal_pos)
            else 2
        )
        smaller, larger = min(smaller, larger), max(smaller, larger)

        return max(smaller, larger - (1 if clock else 0))

    def line_passing_through_goal_can_cut(self, one_pos, two_pos, goal_pos):
        x1, y1 = one_pos
        x2, y2 = two_pos
        x, y = goal_pos
        if x1 < x and x2 < x:
            return True
        if x1 > x and x2 > x:
            return True
        if y1 < y and y2 < y:
            return True
        if y1 > y and y2 > y:
            return True
        return False

    def calc_mht_dist(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return abs(x1 - x2) + abs(y1 - y2)

    def is_on_line_with_goal(self, pos, goal_pos):
        x1, y1 = pos
        x2, y2 = goal_pos
        return x1 == x2 or y1 == y2

    def calculate_min_step(self, clock=True):
        ones = [idx for idx, agent in enumerate(self.agents) if agent.get_type() == 1]
        twos = [idx for idx, agent in enumerate(self.agents) if agent.get_type() == 2]
        ones_pos = [self.agent_positions[i] for i in ones]
        twos_pos = [self.agent_positions[i] for i in twos]

        min_step = 1e9
        for one in ones_pos:
            for two in twos_pos:
                step = self.calculate_min_step_for_two(
                    one, two, self.goal_pos, clock=clock
                )
                if step < min_step:
                    min_step = step
        return min_step

    # Getting a random location in a grid, excluding certain locations
    def get_random_pos(
        self, width: int, height: int, exclude: List[Tuple[int, int]] = []
    ) -> Tuple[int, int]:
        while True:
            position = (
                random.randint(0, width - 1),
                random.randint(0, height - 1),
            )
            if position not in exclude:
                return position


class Grid(Controller, Trainer, GridUtil, Visual, IVisual):
    def __init__(
        self,
        width: int,
        height: int,
        agents: List["Agent"],
        storage: "Storage",
    ):
        super().__init__(storage=storage)
        self.width = width
        self.height = height
        self.min_step = 0
        self.step_count = 1

        self.agents: List["Agent"] = agents
        self.idx: int = 0

        self.set_interactive_tiles()
        self.reorder_for_min_step()

    def set_interactive_tiles(self):
        # Assign goal to set position
        self.goal_pos = self.get_random_pos(self.width, self.height)

        # Assign agents to random positions
        self.agent_positions = [
            self.get_random_pos(self.width, self.height) for _ in self.agents
        ]

    # ----- Core Functions ----- #
    def step(self, is_testing=False):
        if self.has_ended():
            return
        if self.idx >= len(self.agents):
            self.step_count += 1
            self.idx = 0

        agent = self.agents[self.idx]
        observed_state = self.extract_state(self.idx)

        # Disable epsilon exploration when testing or when not being trained
        choose_best = is_testing or not agent.get_type() in self.learners
        action = agent.choose_action(observed_state, choose_best=choose_best)
        reward, next_state, is_terminal = self.move(self.idx, action)

        loss, epsilon = agent.update(
            state=observed_state,
            action=action,
            reward=reward,
            next_state=next_state,
            is_terminal=is_terminal,
        )

        self.idx += 1
        return loss, epsilon

    def move(
        self, idx: int, action: Tuple[int, int]
    ):  # List of actions, in the same order as self.agents
        # Update agent to temporary location according to move
        old_x, old_y = self.agent_positions[idx]
        dx, dy = action
        new_x, new_y = old_x + dx, old_y + dy

        reward = 0

        # Penalise when walking into wall
        def clamp(i: int, lower: int, upper: int) -> Tuple[int, int]:
            penalty = -10 if i < lower or i > upper else 0
            return penalty, min(max(i, lower), upper)

        # Retreive reward and new location according to Entity.interaction
        penalty, new_x = clamp(new_x, 0, self.width - 1)
        penalty, new_y = clamp(new_y, 0, self.height - 1)
        reward += penalty

        # -1 for each turn
        reward -= 1

        new_pos = new_x, new_y
        agent = self.agents[idx]
        if new_pos == self.goal_pos:
            if agent.have_secret:
                # +50 for reaching the goal with secret
                reward += 50
                self.goal_reached = True
        else:
            other_indices = [
                other_idx
                for (other_idx, pos) in enumerate(self.agent_positions)
                if other_idx != idx and pos == new_pos
            ]
            other_agents_diff_type = [
                self.agents[o_idx]
                for o_idx in other_indices
                if self.agents[o_idx].get_type() != self.agents[idx].get_type()
            ]
            if len(other_agents_diff_type) > 0:
                if not self.agents[idx].have_secret:
                    # + 20 for getting the secret
                    reward += 20
                for agent in other_agents_diff_type + [self.agents[idx]]:
                    agent.have_secret_(True)

        self.agent_positions[idx] = new_pos
        return reward, self.extract_state(idx), self.goal_reached

    # ----- Private Functions ----- #
    def reorder_for_min_step(self):
        # Further away an agent is from goal position, earlier it moves
        idx_positions = list(enumerate(self.agent_positions))
        idx_positions.sort(
            key=lambda idx_pos: -self.calc_mht_dist(idx_pos[1], self.goal_pos)
        )
        self.agents = [self.agents[i] for i, pos in idx_positions]
        self.agent_positions = [self.agent_positions[i] for i, pos in idx_positions]

    # ----- Public Functions ----- #
    def reset(self, random_pos=True):
        self.step_count = 1
        self.goal_reached = False
        self.idx = 0
        if random_pos:
            self.set_interactive_tiles()
        self.min_step = self.calculate_min_step()
        for agent in self.agents:
            agent.reset()
        self.reorder_for_min_step()

    def extract_state(self, idx):
        if debug:
            print(f"all agent pos: {self.agent_positions}")
            print(f"all types: {[agent.get_type() for agent in self.agents]}")

        type = self.agents[idx].get_type()
        x, y = self.agent_positions[idx]
        x2, y2 = self.get_closest_other_agent(x, y, type)
        x3, y3 = self.get_goal_positions()

        state = torch.zeros(state_size, dtype=dtype)

        # One-hot vector of agent_pos, closest_oppo, goal_pos concatenatanted, with one bit for self.have_secret
        state[x * side + y] = 1
        state[side**2 + x2 * side + y2] = 1
        state[side**2 * 2 + x3 * side + y3] = 1
        state[side**2 * 3] = 1 if self.agents[idx].have_secret else 0

        return state

    def get_closest_other_agent(self, x, y, type):
        other_type = 2 if type == 1 else 1
        min_dist = 1e9
        min_x, min_y = -1, -1

        for agent, agent_pos in zip(self.agents, self.agent_positions):
            if agent.get_type() == other_type:
                other_x, other_y = agent_pos
                dist = abs(other_x - x) + abs(other_y - y)
                if dist < min_dist:
                    min_x, min_y = other_x, other_y
                    min_dist = dist

        if debug:
            print(
                f"for current agent {x, y}, closest agent of opposite type of {type} is at {min_x, min_y}"
            )
        return min_x, min_y


In [8]:
import random, os, numpy as np, torch

def seed_all(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


In [9]:
max_exp = 100
upd_freq = 80
gamma = 0.9
eps_min = 0.005
batch_size = 32
eps_decay_final_step = 1.99e4
max_grad_norm = 5e3

In [10]:
seed_all(seed=1234)

In [11]:
storage = Storage(20000)
buffer1, buffer2 = ExpBuffer(max=max_exp), ExpBuffer(max=max_exp)
dqn1, dqn2 = DQN(state_size=state_size, action_size=action_size), DQN(state_size=state_size, action_size=action_size)
kwargs = {
    "gamma": gamma,
    "batch_size": batch_size,
}
agents = [
    Agent1(dqn1, buffer1, **kwargs),
    Agent1(dqn1, buffer1, **kwargs),
    Agent2(dqn2, buffer2, **kwargs),
    Agent2(dqn2, buffer2, **kwargs),
]
grid = Grid(
    side,
    side,
    agents,
    storage,
)

We start by training agents with said parameters to demonstrate its performance.

In [ ]:
grid.train(
    20000,
    upd_freq=upd_freq,
    eps_min=eps_min,
    eps_decay_final_step=eps_decay_final_step,
    max_grad_norm=max_grad_norm,
    dqn1=dqn1,
    dqn2=dqn2,
)
# dqn1.save("dqn1")
# dqn2.save("dqn2")
Graph(storage=storage, keys=["ml_losses"]).show()
Graph(storage=storage, keys=["excess_step", "epsilon"]).show()

In [ ]:
grid.small_test(10000)
Graph(storage=storage, keys=["excess_step_hist"]).show()

Visualisation of the trained agents.

In [ ]:
grid.disable_learning(agent_type=1)
grid.disable_learning(agent_type=2)
vis = Visualization(storage, grid).show()

We then load the best performing networks (trained with the correct seed) and run a test of random 10000 scenarrios.

In [ ]:
dqn1.load("dqn1")
dqn2.load("dqn2")

storage.excess_step_hist = Array("i", 51)
grid.small_test(10000)
Graph(storage=storage, keys=["excess_step_hist"]).show()

Lastly we run a full test on all possible scenarios.

In [ ]:
storage.excess_step_hist = Array("i", 51)
grid.test()
Graph(storage=storage, keys=["step_count_hist", "excess_step_hist"]).show()
grid.reset()

## **Game Thoeretic Analysis**

To analyse the scenario in a game-theoretic sense, we simply the grid to be a 1x5 grid and we place only 2 agents. They have the following positions (for simplification, only the x-coordinate is written out):
- Goal: 0
- Agent_1: 1
- Agent_2: 3

| Goal | Agent_1 | [Empty] | Agent_2 | [Empty] |
|:------|------|--------|------|-------:|

And using the reward structure we defined, we have the following payoff matrix, with Agent_1 being the row player and Agent_2 being the column player.

|/| Left | Right | Top | Down | Stay Still |
|:------|------|--------|-----|--------|-------:|
| Left | -1, -1 | -1, -1|-1, -10|-1, -10|-1, -1|
| Right | 20, 20 | -1, -1|-1, -10|-1, -10|-1, -1|
| Top | -10, -1 | -10, -1|-10, -10|-10, -10|-10, -1|
| Down | -10, -1 | -10, -1|-10, -10|-10, -10|-10, -1|
| Stay Still | -1, -1 | -1, -1|-1, -10|-1, -10|-1, -1|


Since for both players, strategy **Top** and **Down** are always domiated by **Stay Still** we simplify by removing the 2 rows/columns

|/| Left | Right | Stay Still |
|:------|------|--------|-------:|
| Left | -1, -1 | -1, -1|-1, -1|
| Right | 20, 20 | -1, -1|-1, -1|
| Stay Still | -1, -1 | -1, -1|-1, -1|

It then becomes clear (Right, Left) is the pareto optimal outcome as well as the nash equilibrium. At the point, neither player has the incentive to change their strategy.

|| Left |
|:---------------|--------:|
| Right | 20, 20 |

If agents were allowed to jump to locations by paying an amount equal to manhattan distance D. This can be generalised into the following form over a longer distance, where D is the manhattan distance bewteen the 2 agents

|/| Meetup | Wait |
|:------|------|-------:|
| Meetup | 20 - D/2, 20 - D/2 | 20 - D, 20 - D|
| Wait | 20 - D, 20 - D | $-\infty$, $-\infty$|

Therefore agents' behaviours are expected to converge to a fair solution where both agents actively move closer and try to meet up at each step. This strategy profile is stable and no agents have the incentive to deviate.

By using Q-learning implementation, we can quickly observe the outcome and be confident that eventually it does converge to proposed strategy. Whereas by using game theory, we can understand the reason being the converged strategy, as well as be certain that such convergence is stable and certain in the long run. Certainty, or proof of correctness, is an aspect covered only by theoretical approach.